In [15]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import re

from transformers import AutoModel, AutoTokenizer
from sentence_transformers import SentenceTransformer
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

# Descargar recursos adicionales de NLTK
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELLPHOTO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELLPHOTO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Cargue de los datos y objetivo

El objetivo de este desafío es agrupar productos similares utilizando algunos atributos de estos como pueden ser el título, la descripción o su imagen para lo que se utilizo TF-IDF y Coseno y BERT y Coseno

In [16]:
# Cargamos los datos de entrenamiento
data_train = pd.read_csv("items_titles.csv")

# Cargamos los datos de prueba
data_test = pd.read_csv("items_titles_test.csv")


In [17]:
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
data_train.head(20)

,ITE_ITEM_TITLE
0,Tênis Ascension Posh Masculino - Preto E Verme...
1,Tenis Para Caminhada Super Levinho Spider Corr...
2,Tênis Feminino Le Parc Hocks Black/ice Origina...
3,Tênis Olympikus Esportivo Academia Nova Tendên...
4,Inteligente Led Bicicleta Tauda Luz Usb Bicicl...
5,Tênis Casual Masculino Zarato 941 Preto 632
6,Tênis Infantil Ortopasso Conforto Jogging
7,Tv Samsung Qled 8k Q800t Semi Nova
8,Tênis Usthemp Short Temático - Maria Vira-lata 2
9,Sapatênis West Coast Urban Couro Masculino


In [18]:
data_test.head(20)

,ITE_ITEM_TITLE
0,Tênis Olympikus Esporte Valente - Masculino Kids
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...
2,Tênis Usthemp Slip-on Temático - Labrador 2
3,Tênis Casual Feminino Moleca Tecido Tie Dye
4,Tênis Star Baby Sapatinho Conforto + Brinde
5,Tênis Oakley Frequency 3.0 Preto/marrom
6,Tênis Jogging Feminino Premium Super Lançament...
7,Under Armour Hovr Phantom 2 Conexão Bluetooth ...
8,Tenis Infantil Feminino Menina Criança Moça
9,Tênis Labellamafia Saturn 6 Cores Disponíveis


## 1. Preprocesamiento de texto

En esta primera fase se realiza un preprocesamiento de los títulos, que consiste en convertirlos a minúsculas, eliminar las stopwords y aplicar stemming.


In [19]:
# Función para corregir todas las ocurrencias de palabras mal escritas para una palabra objetivo
# en una frase, dada un diccionario de palabras mal escritas.

def corregir_todas_las_palabras(frase, palabras_incorrectas, palabra_correcta):
  """
  Corrige todas las ocurrencias de palabras mal escritas para una palabra objetivo
  en una frase, dada un diccionario de palabras mal escritas.

  Args:
      frase: La frase a corregir.
      palabras_incorrectas: Un diccionario con las palabras mal escritas como claves
                          y la palabra correcta como valor.
      palabra_correcta: (Opcional) La palabra correcta para la que se corrigen las variantes.

  Returns:
      La frase corregida.
  """

  # Convertimos la frase a minúsculas
  frase_min = frase.lower()

  # Iteramos sobre cada palabra de la frase
  for i in range(len(frase_min)):
    # Obtenemos la palabra actual
    palabra = frase_min[i]

    # Comprobamos si la palabra actual está en el diccionario de palabras mal escritas
    if palabra in palabras_incorrectas:
      # La corregimos por la palabra correcta
      frase_min = frase_min[:i] + palabra_correcta + frase_min[i + 1:]

  # Devolvemos la frase corregida
  return frase_min

palabras_incorrectas = {
  "tenis": "Tenis",
  "zapatos": "Zapatos",
  "calzado": "Calzado",
  "tenis": "Tenis",
  "teniss": "Tenis",
  "tenisss": "Tenis",
  "teni": "Tenis",
  "tinnis": "Tenis",
  "Tênis": "Tenis",
  "tênis": "Tenis",
  "tênis": "Tenis"
}


## Convertimos los títulos a minúsculas
data_train["ITE_ITEM_TITLE"] = data_train["ITE_ITEM_TITLE"].str.lower()
data_test["ITE_ITEM_TITLE"] = data_test["ITE_ITEM_TITLE"].str.lower()
#
## Eliminamos los espacios en blanco
data_train["ITE_ITEM_TITLE"] = data_train["ITE_ITEM_TITLE"].str.strip()
data_test["ITE_ITEM_TITLE"] = data_test["ITE_ITEM_TITLE"].str.strip()
# Leer la base de datos y seleccionar la variable que contiene las frases

data_train['ITE_ITEM_TITLE'] = data_train['ITE_ITEM_TITLE'].str.strip()
data_train['ITE_ITEM_TITLE'] = data_train["ITE_ITEM_TITLE"].str.lstrip()
data_test['ITE_ITEM_TITLE'] = data_test['ITE_ITEM_TITLE'].str.strip()
data_test['ITE_ITEM_TITLE'] = data_test["ITE_ITEM_TITLE"].str.lstrip()


data_train["ITE_ITEM_TITLE"] = data_train["ITE_ITEM_TITLE"].apply(corregir_todas_las_palabras, palabras_incorrectas=palabras_incorrectas, palabra_correcta="Tenis")
data_test["ITE_ITEM_TITLE"] = data_test["ITE_ITEM_TITLE"].apply(corregir_todas_las_palabras, palabras_incorrectas=palabras_incorrectas, palabra_correcta="Tenis")




# Definir una función para eliminar espacios en blanco al principio y al final usando expresiones regulares
def limpiar_espacios(cadena):
    return re.sub(r'^\s+|\s+$', '', cadena)

# Aplicar la función a la columna 'ITE_ITEM_TITLE'
data_train['ITE_ITEM_TITLE'] = data_train['ITE_ITEM_TITLE'].apply(limpiar_espacios)
data_test['ITE_ITEM_TITLE'] = data_test['ITE_ITEM_TITLE'].apply(limpiar_espacios)
# Mostrar el DataFrame después de la corrección
print(data_train)


                                          ITE_ITEM_TITLE
0      tênis ascension posh masculino - preto e vermelho
1      tenis para caminhada super levinho spider corrida
2      tênis feminino le parc hocks black/ice origina...
3      tênis olympikus esportivo academia nova tendên...
4      inteligente led bicicleta tauda luz usb bicicl...
...                                                  ...
29995  tênis vans old skool i love my vans - usado - ...
29996                tênis feminino preto moleca 5296155
29997  tenis botinha com pelo via marte original lanç...
29998  tênis slip on feminino masculino original sapa...
29999  bicicleta nathor rosa infantil sem pedal balan...

[30000 rows x 1 columns]


In [20]:
data_test.head(20)

,ITE_ITEM_TITLE
0,tênis olympikus esporte valente - masculino kids
1,bicicleta barra forte samy c/ 6 marchas cubo c...
2,tênis usthemp slip-on temático - labrador 2
3,tênis casual feminino moleca tecido tie dye
4,tênis star baby sapatinho conforto + brinde
5,tênis oakley frequency 3.0 preto/marrom
6,tênis jogging feminino premium super lançament...
7,under armour hovr phantom 2 conexão bluetooth ...
8,tenis infantil feminino menina criança moça
9,tênis labellamafia saturn 6 cores disponíveis


## 1.1 TF-IDF y Coseno

A continuación, se aplica la técnica de vectorización TF-IDF a los títulos de entrenamiento y prueba. Esto genera matrices de términos, en las que cada fila representa un documento (título) y cada columna representa una palabra.

La similitud de coseno se calcula a partir de las matrices de términos. Esta medida de similitud mide la similitud entre dos vectores en un espacio multidimensional.

Los títulos con una similitud superior a un valor determinado (0.5 en este caso) se consideran similares. Estos títulos se almacenan en un DataFrame, junto con su puntuación de similitud.

Finalmente, el DataFrame se ordena por puntuación de similitud, de mayor a menor.

In [22]:

# Cargar los datos
df_train = data_train
df_test = data_test

# Preprocesamiento de texto
def preprocess_text(text):
    stop_words = set(stopwords.words('portuguese'))
    stemmer = SnowballStemmer('portuguese')
    
    # Tokenización y eliminación de stop words
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha() and token not in stop_words]
    
    # Aplicar stemming
    tokens = [stemmer.stem(token) for token in tokens]
    
    return ' '.join(tokens)

# Aplicar preprocesamiento a los títulos
df_train['processed_title'] = df_train['ITE_ITEM_TITLE'].apply(preprocess_text)
df_test['processed_title'] = df_test['ITE_ITEM_TITLE'].apply(preprocess_text)

# Vectorización de texto usando TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix_train = vectorizer.fit_transform(df_train['processed_title'])
tfidf_matrix_test = vectorizer.transform(df_test['processed_title'])

# Calcular similitud de coseno
cosine_similarities = cosine_similarity(tfidf_matrix_test, tfidf_matrix_train)

# Definir un umbral de similitud
threshold = 0.5

# Crear un DataFrame para almacenar los resultados
results = []


# Iterar sobre los títulos de prueba y encontrar títulos similares en el conjunto de entrenamiento
for i, test_title in enumerate(df_test['ITE_ITEM_TITLE']):
    similar_titles = df_train[cosine_similarities[i] > threshold]['ITE_ITEM_TITLE'].tolist()
    
    # Calcular el score de similitud
    similarity_scores = cosine_similarities[i][cosine_similarities[i] > threshold]
    
    # Escalar a un rango de 0 a 1
    scaled_scores = (similarity_scores + 1) / 2
    
    # Agregar cada par (título de prueba, título similar, score de similitud) como una nueva fila
    for similar_title, score in zip(similar_titles, scaled_scores):
        results.append({'Train Title': test_title, 'Test Title': similar_title, 'Similarity Score': score})

        # Crear un DataFrame de resultados
df_results = pd.DataFrame(results)
df_results_sorted = df_results.sort_values(by='Similarity Score', ascending=False)

# Guardar los resultados en un archivo CSV
#df_results.to_csv('similarity_results.csv', index=False)


In [23]:
df_results_sorted.head(100)

,Train Title,Test Title,Similarity Score
269974,tênis casual kolosh feminino c2761,tênis casual feminino kolosh c1282,1.0
87048,tênis kolosh casual feminino - c1287,tênis casual feminino kolosh c1282,1.0
2924,tênis qix skate combat iv camurça preto branco...,tênis qix skate combat iv camurça preto branco...,1.0
270058,tênis casual kolosh feminino c2761,tênis kolosh c1629 feminino casual,1.0
87063,tênis kolosh casual feminino - c1287,tênis feminino kolosh k8681 casual,1.0
...,...,...,...
222712,tênis infantil led roller celebration de luz b...,tênis infantil led roller celebration de luz b...,1.0
222779,tênis polo farm 921 masculino - 2,tênis polo farm 275 masculino,1.0
222852,sapatenis masculino barato,sapatenis masculino barato,1.0
13440,tênis feminino casual leve dia a dia macio mod...,tênis feminino casual leve dia a dia macio mod...,1.0


In [24]:
df_results_sorted.tail(100)

,Train Title,Test Title,Similarity Score
110548,tenis feminino chunky elastico tie dye,tênis slip on tie dye amarelo,0.750021
156080,tênis meia feminino masculino slip on academia...,tenis meias modare casual feminino slip on con...,0.750020
221553,sneaker chunky feminino pedraria removível cad...,tênis pedraria chunky sneaker feminino lançam...,0.750020
221551,sneaker chunky feminino pedraria removível cad...,tênis chunky pedraria sneaker feminino lançame...,0.750020
193307,tenis masculino para academia e caminhada,tenis activitta academia caminhada sem cadarço...,0.750019
...,...,...,...
60182,sapatênis colcci kelson em couro branco - masc...,sapatênis colcci lettering off-witte original,0.750001
152877,tênis masculino creation 20 caminhada envio im...,tênis wave creation 21,0.750001
203993,sapatenis tenis casual masculino solado reto r...,tênis sapatênis vermelho masculino + relógio d...,0.750000
147143,453072 tenis mini walk branco/marinho/vermelho...,tenis klin 09/2021 256.063 preto/goma,0.750000


## 1.2 BERT y Coseno


Se utiliza modelo de lenguaje pre-entrenado que aprende representaciones vectoriales de palabras y frases y una medida de similitud entre vectores que mide el ángulo entre ellos.

In [27]:


def calculate_similarity(emb1, emb2):
    return np.dot(emb1.flatten(), emb2.flatten()) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))

titulos_train = df_train['ITE_ITEM_TITLE'].sample(n=1000, random_state=42)
titulos_test = df_test['ITE_ITEM_TITLE'].sample(n=1000, random_state=42)

tokenized_titles_train = [" ".join(word_tokenize(titulo.lower())) for titulo in titulos_train]
tokenized_titles_test = [" ".join(word_tokenize(titulo.lower())) for titulo in titulos_test]

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

title_embeddings_train = []
title_embeddings_test = []

for titulo in tokenized_titles_train:
    inputs = tokenizer(titulo, return_tensors="pt")
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    title_embeddings_train.append(embeddings)

for titulo in tokenized_titles_test:
    inputs = tokenizer(titulo, return_tensors="pt")
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    title_embeddings_test.append(embeddings)

# Calcular similitudes y almacenar en un DataFrame
results = []

for emb1, titulo1 in zip(title_embeddings_train, titulos_train):
    for emb2, titulo2 in zip(title_embeddings_test, titulos_test):
        similarity = calculate_similarity(emb1, emb2)
        results.append({'Train Title': titulo1, 'Test Title': titulo2, 'Similarity Score': similarity})

# Crear DataFrame y ordenar por Similarity de mayor a menor
df_results = pd.DataFrame(results)
df_results_sorted = df_results.sort_values(by='Similarity Score', ascending=False)

# Mostrar el DataFrame ordenado
#print(df_results_sorted)


In [28]:
df_results_sorted.head(10)

,Train Title,Test Title,Similarity Score
534653,bicicleta feminina aro 26 na cor creme com cer...,bicicleta feminina aro 26 na cor creme com cer...,1.000000
321007,sapato infantil klin outdoor 161 - 1741602011,sapato infantil klin outdoor 161 - 1741602011,1.000000
527969,"tênis militar tático, esporte, caminhada, resi...","tênis militar tático, esporte, caminhada, resi...",1.000000
802060,tênis caminhada six street lunar - white green...,tênis caminhada six street lunar - white green...,1.000000
553300,tenis feminino masculino numero grande 38 39 4...,tenis feminino masculino numero grande 38 39 4...,1.000000
447427,bicicleta aro 16 com rodinhas feminina ultra kids,bicicleta com rodinhas aro 16 feminina ultra kids,0.993038
521048,tenis infantil masculino menino escolar c revenda,tenis infantil masculino menino escolar d revenda,0.985190
621459,sapatenis masculino em couro legítimo confort...,sapatenis masculino confortavel em couro legitimo,0.979148
382409,tênis usthemp half temático - chihuahua pelo ...,tênis usthemp move temático - chihuahua pelo ...,0.972723
172135,sapatenis feminino rosa confortável - compre a...,sapatenis feminino rosa cano baixo - compre ag...,0.972493
